# Python function to match the location of air pollution levels to location of UK Biobank participants

**Author:** Yizhou Yu
**Affiliation:** MRC Toxicology Unit, University of Cambridge

Date: 7 May 2020

In this python script, I use a function in sklearn to find the nearest modelled air pollutant value based longitude and latitude values. I then asign this value to each UK Biobank participant in the COVID-19 subset. The access to the UK Biobank data is granted under the Application ID 60124. 

Note. I purposefully do not show any part of the dataset since the UK Biobank contains sensitive data. Access to this data needs to be applived via ukbiobank.co.uk.


### First try this with PM2.5

In [1]:
import pandas as pd

In [5]:
ukb_covid_df = pd.read_csv("data_v4/ukb_covid_lonlat_df.csv", usecols=['ukb_lon','ukb_lat','eid'])

In [6]:
pm25_df = pd.read_csv("data_v4/processed_pm25_lonlat.csv", usecols=['pm25_lon','pm25_lat','pm25_val'])

Use this script, which should be 100% correct... https://automating-gis-processes.github.io/site/develop/notebooks/L3/nearest-neighbor-faster.html

In [3]:
#build functions
from sklearn.neighbors import BallTree
import numpy as np

def get_nearest(src_points, candidates, k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='haversine')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # Get closest indices and distances (i.e. array at index 0)
    # note: for the second closest points, you would take index 1, etc.
    closest = indices[0]
    closest_dist = distances[0]

    # Return indices and distances
    return (closest, closest_dist)


def nearest_neighbor(left_gdf, right_gdf, return_dist=False):
    """
    For each point in left_gdf, find closest point in right GeoDataFrame and return them.

    NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
    """

    left_geom_col = left_gdf.geometry.name
    right_geom_col = right_gdf.geometry.name

    # Ensure that index in right gdf is formed of sequential numbers
    right = right_gdf.copy().reset_index(drop=True)

    # Parse coordinates from points and insert them into a numpy array as RADIANS
    left_radians = np.array(left_gdf[left_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())
    right_radians = np.array(right[right_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())

    # Find the nearest points
    # -----------------------
    # closest ==> index in right_gdf that corresponds to the closest point
    # dist ==> distance between the nearest neighbors (in meters)

    closest, dist = get_nearest(src_points=left_radians, candidates=right_radians)

    # Return points from right GeoDataFrame that are closest to points in left GeoDataFrame
    closest_points = right.loc[closest]

    # Ensure that the index corresponds the one in left_gdf
    closest_points = closest_points.reset_index(drop=True)

    # Add distance if requested
    if return_dist:
        # Convert to meters from radians
        earth_radius = 6371000  # meters
        closest_points['distance'] = dist * earth_radius

    return closest_points

In [31]:
# Fit data into these functions: have a column called geometry
import geopandas as gpd

#pm25_coord_list = list(zip(pm25_df['pm25_lon'],pm25_df['pm25_lat']))
#covid_coord_list = list(zip(ukb_covid_df['ukb_lon'],ukb_covid_df['ukb_lat']))

#pm25_df['geometry'] = pm25_coord_list
#ukb_covid_df['geometry'] = covid_coord_list

pm25_gdf = gpd.GeoDataFrame(
    pm25_df, geometry=gpd.points_from_xy(pm25_df.pm25_lon, pm25_df.pm25_lat))

ukb_covid_gdf = gpd.GeoDataFrame(
    ukb_covid_df, geometry=gpd.points_from_xy(ukb_covid_df.ukb_lon, ukb_covid_df.ukb_lat))

In [32]:
# Note: haversine distance which is implemented here is a bit slower than using e.g. 'euclidean' metric
# but useful as we get the distance between points in meters
matched_df = nearest_neighbor(ukb_covid_gdf, pm25_gdf, return_dist=True)

In [33]:
# Rename the geometry of closest stops gdf so that we can easily identify it
matched_df = matched_df.rename(columns={'geometry': 'pm25_geom'})

# Merge the datasets by index (for this, it is good to use '.join()' -function)
ukb_pm25_df = ukb_covid_gdf.join(matched_df)

In [34]:
#get rmsd
ukb_pm25_df['distance'].describe()

count    1464.000000
mean      962.196857
std        21.434083
min       930.406112
25%       939.639609
50%       966.833002
75%       973.662456
max      1001.337192
Name: distance, dtype: float64

In [34]:
#this looks good!

ukb_pm25_df.to_csv('data_output_v4/merged_ukb_pm25.csv', index=False)

## I still need to generate a dataset of UK population density

-> get the city names of the ukb geo coordinate

In [35]:
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
#create a locator function
locator = Nominatim(user_agent="test", timeout=10)
#add a rate limiter to prevent server overflow 
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)
# do a test
coordinates = '53.480837, -2.244914'
location = locator.reverse(coordinates)
location.raw.get('city')

In [29]:
location.raw['address']

NameError: name 'location' is not defined

In [39]:
ukb_covid_df['locator_input'] = ukb_covid_df['ukb_lat'].map(str) + " , " + ukb_covid_df['ukb_lon'].map(str)
ukb_covid_df['locator_input'].head()

0       53.3707658704288 , -2.15175829308345
1     55.041934472539396 , -1.62596878710592
2     51.3770102204939 , -0.0616916291513967
3       53.4966856530176 , -2.06176460094776
4    51.67872198359771 , -1.3071806496557299
Name: locator_input, dtype: object

In [ ]:
#ukb_covid_df['city'] = ukb_covid_df['locator_input'].apply(get_city).raw['address']['city']
#ukb_covid_df

ukb_covid_df['address'] = ukb_covid_df['locator_input'].apply(rgeocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('53.3707658704288 , -2.15175829308345',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/urllib/request.py", line 1319, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1230, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1276, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1225, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1004, in _send_output
    self.send(msg)
  F

RateLimiter caught an error, retrying (0/2 tries). Called with (*('55.041934472539396 , -1.62596878710592',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/urllib/request.py", line 1319, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1230, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1276, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1225, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1004, in _send_output
    self.send(msg)
 

RateLimiter caught an error, retrying (0/2 tries). Called with (*('51.3770102204939 , -0.0616916291513967',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/urllib/request.py", line 1319, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1230, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1276, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1225, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1004, in _send_output
    self.send(msg)
 

RateLimiter caught an error, retrying (0/2 tries). Called with (*('53.4966856530176 , -2.06176460094776',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/urllib/request.py", line 1319, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1230, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1276, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1225, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1004, in _send_output
    self.send(msg)
  F

RateLimiter caught an error, retrying (0/2 tries). Called with (*('51.67872198359771 , -1.3071806496557299',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/urllib/request.py", line 1319, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1230, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1276, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1225, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1004, in _send_output
    self.send(msg)


...

RateLimiter caught an error, retrying (0/2 tries). Called with (*('54.5544769122655 , -1.1820512805253',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/urllib/request.py", line 1319, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1230, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1276, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1225, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1004, in _send_output
    self.send(msg)
  Fi

RateLimiter caught an error, retrying (0/2 tries). Called with (*('53.3693576499988 , -1.4003022534954301',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/urllib/request.py", line 1319, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1230, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1276, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1225, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1004, in _send_output
    self.send(msg)
 

RateLimiter caught an error, retrying (0/2 tries). Called with (*('52.9823759917877 , -1.31630494989398',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/urllib/request.py", line 1319, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1230, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1276, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1225, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1004, in _send_output
    self.send(msg)
  F

RateLimiter caught an error, retrying (0/2 tries). Called with (*('53.586030674793605 , -2.3640103387804303',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/urllib/request.py", line 1319, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1230, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1276, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1225, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1004, in _send_output
    self.send(msg)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('53.3762000919677 , -2.9334204751859096',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/urllib/request.py", line 1319, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1230, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1276, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1225, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1004, in _send_output
    self.send(msg)
 

In [10]:
def loc_specific_area(address):
    if address.raw['address'].get('city') == None:
        city_town = address.raw['address'].get('town')
        if city_town == None:
            city_town = address.raw['address'].get('village')
    else: 
        city_town = address.raw['address'].get('city')
    return city_town

def loc_gen_area(address):
    if address.raw['address'].get('district') == None:
        area = address.raw['address'].get('county')
        if address.raw['address'].get('county') == None:
            area = address.raw['address'].get('state_district')
    else:
        area = address.raw['address'].get('district')
    return area

time: 5.58 ms


In [267]:
ukb_covid_df['spec_area'] = ukb_covid_df['address'].apply(loc_specific_area)
ukb_covid_df['gen_area'] = ukb_covid_df['address'].apply(loc_gen_area)

,ukb_lon,ukb_lat,eid,geometry,locator_input,address,city_town,spec_area,gen_area
0,-2.151758,53.370766,1007104,POINT (-2.15176 53.37077),"53.3707658704288 , -2.15175829308345","(Kitt's Moss, Stockport, Greater Manchester, N...",Stockport,Stockport,Greater Manchester
1,-1.625969,55.041934,1011824,POINT (-1.62597 55.04193),"55.041934472539396 , -1.62596878710592","(Ashwood Grove, North Tyneside, Tyne and Wear,...",North Tyneside,North Tyneside,Tyne and Wear
2,-0.061692,51.377010,1012827,POINT (-0.06169 51.37701),"51.3770102204939 , -0.0616916291513967","(180, Shirley Road, Addiscombe, London Borough...",London Borough of Croydon,London Borough of Croydon,Greater London
3,-2.061765,53.496686,1017007,POINT (-2.06176 53.49669),"53.4966856530176 , -2.06176460094776","(Stephens Close, Springs, Tameside, Greater Ma...",Tameside,Tameside,Greater Manchester
4,-1.307181,51.678722,1018400,POINT (-1.30718 51.67872),"51.67872198359771 , -1.3071806496557299","(Prince of Wales (PH) carpark, Barrow Road, St...",Oxfordshire,Shippon,Oxfordshire
...,...,...,...,...,...,...,...,...,...
1459,-1.475664,53.351735,6004019,POINT (-1.47566 53.35173),"53.35173484754729 , -1.4756643399070901","(Norton Lees Road, Meersbrook, Sheffield, York...",Sheffield,Sheffield,Yorkshire and the Humber
1460,-0.896758,51.306919,6006093,POINT (-0.89676 51.30692),"51.3069192727867 , -0.896757576628775","(Oakland Terrace, Hartley Wintney, Hart, Hamps...",Hart,Hart,Hampshire
1461,-2.549148,51.526625,6015075,POINT (-2.54915 51.52662),"51.526624894285405 , -2.54914800578694","(34, Elm Close, Stoke Gifford, South Glouceste...",South Gloucestershire,Stoke Gifford,South Gloucestershire
1462,-2.592040,51.499442,6016074,POINT (-2.59204 51.49944),"51.4994415400276 , -2.59203951292197","(Southmead Hospital, Southmead Road, Golden Hi...",Bristol,Bristol,City of Bristol


time: 36.7 ms


In [262]:
locator.reverse('51.5451430049752 , -2.4195862989520402').raw['address']

{'house_number': '2',
 'road': 'Tyndale Avenue',
 'suburb': 'Yate',
 'town': 'Yate',
 'county': 'South Gloucestershire',
 'state_district': 'South West England',
 'state': 'England',
 'postcode': 'BS37 5EU',
 'country': 'United Kingdom',
 'country_code': 'gb'}

time: 224 ms


This shows that there are no more "None" values

In [270]:
ukb_covid_df.to_csv("data_output_v4/2_5_2020_ukb_eid_match_cities.csv", index=False)

time: 74.7 ms


## Match UKB eids to additional air data

### NO2

3 May 2020

In [18]:
#read data and delete NAs
no2_df = pd.read_csv('data_v4/processed_no2_lonlat.csv', usecols = ['no2_lon','no2_lat','no2_val'])
no2_df.head()

,no2_lon,no2_lat,no2_val
0,-0.888372,60.853691,1.734648
1,-0.907079,60.844866,1.748184
2,-0.888684,60.844715,1.756501
3,-0.870288,60.844562,1.764046
4,-0.851893,60.844406,1.807779


In [23]:
# use the previous functions to 1. translate to long lat in R, then match + calculate nearest distance here. 

import geopandas as gpd
#prep df 
no2_gdf = gpd.GeoDataFrame(
    no2_df, geometry=gpd.points_from_xy(no2_df.no2_lon, no2_df.no2_lat))

#match 
no2_matched_df = nearest_neighbor(ukb_covid_gdf, no2_gdf, return_dist=True)

#rename col name
no2_matched_df = no2_matched_df.rename(columns={'geometry': 'no2_geom'})

,no2_lon,no2_lat,no2_val,no2_geom,distance
0,-2.159256,53.366262,13.69212,POINT (-2.15926 53.36626),972.410665
1,-1.618187,55.037417,12.87829,POINT (-1.61819 55.03742),1000.378115
2,-0.054702,51.372398,17.08413,POINT (-0.05470 51.37240),931.125832
3,-2.069294,53.492187,17.17420,POINT (-2.06929 53.49219),975.122618
4,-1.300019,51.674184,14.98909,POINT (-1.30002 51.67418),942.681270


time: 6.19 s


In [24]:
#join

no2_ukb_df = ukb_covid_gdf.join(no2_matched_df)

#save 
no2_ukb_df.to_csv('data_output_v4/merged_ukb_no2.csv', index=False)

time: 99.3 ms


### O3

In [6]:
#read data and delete NAs
o3_df = pd.read_csv('data_v4/processed_o3_lonlat.csv', usecols = ['o3_lon','o3_lat','o3_val'])

,o3_lon,o3_lat,o3_val
0,-0.888372,60.853691,4.699172
1,-0.907079,60.844866,4.701599
2,-0.888684,60.844715,4.697112
3,-0.870288,60.844562,4.696401
4,-0.851893,60.844406,4.692276
...,...,...,...
254873,-6.365957,49.885427,10.568590
254874,-6.352077,49.885950,10.562430
254875,-6.338197,49.886471,10.554920
254876,-6.324316,49.886991,10.552480


In [20]:
#prep df 
o3_gdf = gpd.GeoDataFrame(
    o3_df, geometry=gpd.points_from_xy(o3_df.o3_lon, o3_df.o3_lat))

#match 
o3_matched_df = nearest_neighbor(ukb_covid_gdf, o3_gdf, return_dist=True)

#rename col name
o3_matched_df = o3_matched_df.rename(columns={'geometry': 'o3_geom'})
# And the result looks like ...
o3_ukb_df = ukb_covid_gdf.join(o3_matched_df)

,ukb_lon,ukb_lat,eid,geometry,o3_lon,o3_lat,o3_val,o3_geom,distance
0,-2.151758,53.370766,1007104,POINT (-2.15176 53.37077),-2.159256,53.366262,3.690959,POINT (-2.15926 53.36626),972.410665
1,-1.625969,55.041934,1011824,POINT (-1.62597 55.04193),-1.618187,55.037417,3.307217,POINT (-1.61819 55.03742),1000.378115
2,-0.061692,51.377010,1012827,POINT (-0.06169 51.37701),-0.054702,51.372398,10.325390,POINT (-0.05470 51.37240),931.125832
3,-2.061765,53.496686,1017007,POINT (-2.06176 53.49669),-2.069294,53.492187,3.032661,POINT (-2.06929 53.49219),975.122618
4,-1.307181,51.678722,1018400,POINT (-1.30718 51.67872),-1.300019,51.674184,10.100590,POINT (-1.30002 51.67418),942.681270


time: 6.81 s


In [21]:
#save 
o3_ukb_df.to_csv('data_v4/merged_ukb_o3.csv', index=False)

time: 98.3 ms


### PM 10



In [21]:
#read data and delete NAs
pm10_df = pd.read_csv('data_v4/processed_pm10_lonlat.csv', usecols = ['pm10_lon','pm10_lat','pm10_val'])

,pm10_lon,pm10_lat,pm10_val
0,-0.888372,60.853691,4.874836
1,-0.907079,60.844866,4.868680
2,-0.888684,60.844715,4.877372
3,-0.870288,60.844562,4.878715
4,-0.851893,60.844406,4.880319
...,...,...,...
254872,-6.365957,49.885427,8.299677
254873,-6.352077,49.885950,8.310838
254874,-6.338197,49.886471,8.319454
254875,-6.324316,49.886991,8.314877


In [32]:
#prep df 
pm10_gdf = gpd.GeoDataFrame(
    pm10_df, geometry=gpd.points_from_xy(pm10_df.pm10_lon, pm10_df.pm10_lat))

#match 
pm10_matched_df = nearest_neighbor(ukb_covid_gdf, pm10_gdf, return_dist=True).rename(columns={'geometry': 'pm10_geom'})

# And the result looks like ...
pm10_ukb_df = ukb_covid_gdf.join(pm10_matched_df)

,ukb_lon,ukb_lat,eid,geometry,pm10_lon,pm10_lat,pm10_val,pm10_geom,distance
0,-2.151758,53.370766,1007104,POINT (-2.15176 53.37077),-2.159256,53.366262,10.53294,POINT (-2.15926 53.36626),972.410665
1,-1.625969,55.041934,1011824,POINT (-1.62597 55.04193),-1.618187,55.037417,11.06206,POINT (-1.61819 55.03742),1000.378115
2,-0.061692,51.377010,1012827,POINT (-0.06169 51.37701),-0.054702,51.372398,15.89615,POINT (-0.05470 51.37240),931.125832
3,-2.061765,53.496686,1017007,POINT (-2.06176 53.49669),-2.069294,53.492187,11.68648,POINT (-2.06929 53.49219),975.122618
4,-1.307181,51.678722,1018400,POINT (-1.30718 51.67872),-1.300019,51.674184,16.37783,POINT (-1.30002 51.67418),942.681270


time: 6.3 s


In [33]:
#save 
pm10_ukb_df.to_csv('data_output_v4/merged_ukb_pm10.csv', index=False)

time: 113 ms


### SO2



In [24]:
#read data and delete NAs
so2_df = pd.read_csv('data_v4/processed_so2_lonlat.csv')
so2_df.head()

,Unnamed: 0,so2_lon,so2_lat,so2_val
0,1,-0.888372,60.853691,0.194088
1,2,-0.907079,60.844866,0.195659
2,3,-0.888684,60.844715,0.195928
3,4,-0.870288,60.844562,0.196462
4,5,-0.851893,60.844406,0.198325


In [37]:
#prep df 
so2_gdf = gpd.GeoDataFrame(
    so2_df, geometry=gpd.points_from_xy(so2_df.so2_lon, so2_df.so2_lat))

#match 
so2_matched_df = nearest_neighbor(ukb_covid_gdf, so2_gdf, return_dist=True).rename(columns={'geometry': 'so2_geom'})

# And the result looks like ...
so2_ukb_df = ukb_covid_gdf.join(so2_matched_df)

,ukb_lon,ukb_lat,eid,geometry,Unnamed: 0,so2_lon,so2_lat,so2_val,so2_geom,distance
0,-2.151758,53.370766,1007104,POINT (-2.15176 53.37077),138282,-2.159256,53.366262,1.489888,POINT (-2.15926 53.36626),972.410665
1,-1.625969,55.041934,1011824,POINT (-1.62597 55.04193),89102,-1.618187,55.037417,0.872946,POINT (-1.61819 55.03742),1000.378115
2,-0.061692,51.377010,1012827,POINT (-0.06169 51.37701),221729,-0.054702,51.372398,1.173344,POINT (-0.05470 51.37240),931.125832
3,-2.061765,53.496686,1017007,POINT (-2.06176 53.49669),134929,-2.069294,53.492187,2.423522,POINT (-2.06929 53.49219),975.122618
4,-1.307181,51.678722,1018400,POINT (-1.30718 51.67872),211425,-1.300019,51.674184,1.168478,POINT (-1.30002 51.67418),942.681270


time: 6.92 s


In [38]:
#save 
so2_ukb_df.to_csv('data_output_v4/merged_ukb_so2.csv', index=False)

time: 82.8 ms


### NOx



In [25]:
#read data and delete NAs
nox_df = pd.read_csv('data_v4/processed_nox_lonlat.csv')
nox_df.head()

,Unnamed: 0,nox_lon,nox_lat,nox_val
0,1,-0.888372,60.853691,2.110115
1,2,-0.907079,60.844866,2.126862
2,3,-0.888684,60.844715,2.137155
3,4,-0.870288,60.844562,2.146492
4,5,-0.851893,60.844406,2.200650


In [40]:
#prep df 
nox_gdf = gpd.GeoDataFrame(
    nox_df, geometry=gpd.points_from_xy(nox_df.nox_lon, nox_df.nox_lat))

#match 
nox_matched_df = nearest_neighbor(ukb_covid_gdf, nox_gdf, return_dist=True).rename(columns={'geometry': 'nox_geom'})

# And the result looks like ...
nox_ukb_df = ukb_covid_gdf.join(nox_matched_df)

,ukb_lon,ukb_lat,eid,geometry,Unnamed: 0,nox_lon,nox_lat,nox_val,nox_geom,distance
0,-2.151758,53.370766,1007104,POINT (-2.15176 53.37077),138282,-2.159256,53.366262,18.56765,POINT (-2.15926 53.36626),972.410665
1,-1.625969,55.041934,1011824,POINT (-1.62597 55.04193),89102,-1.618187,55.037417,17.21799,POINT (-1.61819 55.03742),1000.378115
2,-0.061692,51.377010,1012827,POINT (-0.06169 51.37701),221729,-0.054702,51.372398,23.94663,POINT (-0.05470 51.37240),931.125832
3,-2.061765,53.496686,1017007,POINT (-2.06176 53.49669),134929,-2.069294,53.492187,24.16020,POINT (-2.06929 53.49219),975.122618
4,-1.307181,51.678722,1018400,POINT (-1.30718 51.67872),211425,-1.300019,51.674184,20.58096,POINT (-1.30002 51.67418),942.681270


time: 6.17 s


In [41]:
#save 
nox_ukb_df.to_csv('data_output_v4/merged_ukb_nox.csv', index=False)

time: 98.6 ms


# Repeat this process for 5-year-average air pollution values

In [ ]:
# I will overwrite on the same variables for simplicity

pm25_df = pd.read_csv("data_output_v4/processed_allAP_lonlat.csv", usecols=['lon','lat','pm25_5yAvg'])
no2_df = pd.read_csv('data_output_v4/processed_allAP_lonlat.csv', usecols = ['lon','lat','no2_5yAvg'])
o3_df = pd.read_csv('data_output_v4/processed_allAP_lonlat.csv', usecols = ['lon','lat','o3_5yAvg'])
pm10_df = pd.read_csv('data_output_v4/processed_allAP_lonlat.csv', usecols = ['lon','lat','pm10_5yAvg'])
# so2_df = pd.read_csv('data_output_v4/processed_allAP_lonlat.csv', usecols = ['lon','lat','so2_val'])
nox_df = pd.read_csv('data_output_v4/processed_allAP_lonlat.csv', usecols = ['lon','lat','nox_5yAvg'])


### References

Note: These packages do not contain a specific citation. I label with the official documentation.

pandas: Data structures for statistical computing in python, McKinney, Proceedings of the 9th Python in Science Conference, Volume 445, 2010.

geopandas: https://geopandas.org/

geopy: https://geopy.readthedocs.io/en/stable/

